In [41]:
from models.DistMult import DistMult
from utils.loaders import load_data
import pickle
import torch
import numpy as np
import random

In [96]:
with open(r'C:\Users\odhra\Documents\graph_project\allosteric_reasonings\data\processed\triplets.pickle', 'rb') as handle:
    data = pickle.load(handle)

with open(r'C:\Users\odhra\Documents\graph_project\allosteric_reasonings\data\processed\triplet_lookup.pickle', 'rb') as handle:
    lookup = pickle.load(handle)

with open(r'C:\Users\odhra\Documents\graph_project\allosteric_reasonings\data\processed\ASD_dictionary.pickle', 'rb') as handle:
    ASD_dictionary = pickle.load(handle)

entities = len(lookup)

In [98]:
unique_elements, counts_elements = np.unique(data, return_counts=True)
print("Frequency of unique values of the said array:")
np.asarray((unique_elements, counts_elements))

Frequency of unique values of the said array:


array([[    0,     1,     2, ..., 70548, 70549, 70550],
       [34324, 55096, 58973, ...,     9,     1,     2]], dtype=int64)

In [104]:
lookup[]

NameError: name 'ASD01910968' is not defined

In [103]:
ASD_dictionary[ASD01910968]

NameError: name 'ASD01910968' is not defined

In [105]:
lookup[ASD01910211]

NameError: name 'ASD01910211' is not defined

In [99]:
number_of_batches = 50
number_of_epochs = 100
random.shuffle(data)
test_data = data[:100] #just limit data to the first 100 rows

In [100]:
all_objects, all_relationships, all_subjects = np.split(test_data, 3, axis=1)
# all_objects = torch.LongTensor(all_objects)
# all_subjects = torch.squeeze(torch.LongTensor(all_subjects))
# all_relationships = torch.LongTensor(all_relationships)

In [102]:
unique_elements, counts_elements = np.unique(all_subjects, return_counts=True)
print("Frequency of unique values of the said array:")
np.asarray((unique_elements, counts_elements))

Frequency of unique values of the said array:


array([[23457, 33399, 40874, 50616, 51051, 60379, 62738],
       [    3,    11,    10,    10,    20,     3,    43]], dtype=int64)

In [91]:
random.shuffle(data)
test_data = data[:100]
all_objects, all_relationships, all_subjects = np.split(test_data, 3, axis=1)
unique_elements, counts_elements = np.unique(all_relationships, return_counts=True)
print("Frequency of unique values of the said array:")
np.asarray((unique_elements, counts_elements))

Frequency of unique values of the said array:


array([[  2],
       [100]], dtype=int64)

In [34]:
objects, subjects, relationships  = load_data(test_data, number_of_batches)
torch.LongTensor(objects[0])

tensor([[43189],
        [29986],
        [30351],
        [38248],
        [65628],
        [41173],
        [29802],
        [46461],
        [30351],
        [60693]])

In [12]:
model = DistMult(num_entities = entities, embedding_dim=100)
print(model.parameters())

<generator object Module.parameters at 0x000001EC9B9B6CF0>


In [ ]:
model.zero_grad()

print('emb_grad')
print(model.emb_e.weight.grad)

loss = model.loss(pred, subj)
loss.backward()

print('emb_grad')
print(model.emb_e.weight.grad)

In [38]:
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)
for epoch in range(number_of_epochs):

        model.train() #tells pytorch you are training rather than evaluating, so dropout happens

        objects, subjects, relationships  = load_data(test_data, number_of_batches)

        for index in range(number_of_batches):

            optimiser.zero_grad()

            obj = torch.LongTensor(objects[index])
            rel = torch.LongTensor(relationships[index])
            subj = torch.squeeze(torch.LongTensor(subjects[index]))

            pred = model.forward(e1 = obj, rel = rel)

            loss = model.loss(pred, subj)

            loss.backward()

            optimiser.step()


        model.eval()

        outputs = model.forward(e1 = all_objects, rel = all_relationships)
        _, predicted = torch.max(outputs.data, 1)

        print('number of correct answers is...') 
        print((predicted == all_subjects).sum().item())



number of correct answers is...
0
number of correct answers is...
2
number of correct answers is...
11
number of correct answers is...
40
number of correct answers is...
61
number of correct answers is...
76
number of correct answers is...
82
number of correct answers is...
84
number of correct answers is...
84
number of correct answers is...
84
number of correct answers is...
84
number of correct answers is...
84
number of correct answers is...
84
number of correct answers is...
84
number of correct answers is...
84


KeyboardInterrupt: 

In [ ]:
# best way to test your system is to train on 100 examples and tests on the 100 same examples - if your model doesn't overfit after a bundle of epochs there is a mistake. 
# knowledge base models take a long long time to converge. They can take 100s of epochs to converge. YOU SHOULD SHUFFLE YOUR DATA EVERY TIME!!! Don't have the same stuff in each batch. 
# learn to use the data loader for shuffling. Alternatively do it yourself.  

In [ ]:
# with BCE you can do label smoothing. 

In [ ]:
out.size()
#okay this makes sense its spitting out a prediction!!! 

In [ ]:
opt = torch.optim.Adam(model.parameters(), lr=0.003, weight_decay=0.995)

In [ ]:
model.train()